In [3]:
import cv2
bright = cv2.imread('cube1.jpg')
dark = cv2.imread('cube8.jpg')

In [7]:
#observation : significant perceptual non-uniformity
#mixing of chrominance (color related information) and luminance (intensity related information) data

#Lab color-space
#L-lightness(intensity) ; independent of color info and encodes brightness only
#a-color component ranging from green to magenta ; encode color
#b-color component ranging from blue to yellow ; encode color

# it has the following properties
# perceptually uniform color space which approximates how we perceive color
# Independent of device ( capturing or displaying)
# Used extensively in Adobe Photoshop
# Is related to the RGB color space by a complex transformation equation

In [4]:
#python
brightLAB = cv2.cvtColor(bright, cv2.COLOR_BGR2LAB)
darkLAB = cv2.cvtColor(dark, cv2.COLOR_BGR2LAB)

In [1]:
#observation
# Similar observations as LAB can be made for Intensity and color components with regard to Illumination changes.
# Perceptual difference between Red and Orange is less even in the outdoor image as compared to LAB.
# White has undergone change in all 3 components.

#HSV color space
# H – Hue ( Dominant Wavelength ).
# S – Saturation ( Purity / shades of the color ).
# V – Value ( Intensity ).

# Best thing is that it uses only one channel to describe color (H), making it very intuitive to specify color.
# Device dependent.

In [5]:
brightHSV = cv2.cvtColor(bright, cv2.COLOR_BGR2HSV)
darkHSV = cv2.cvtColor(dark, cv2.COLOR_BGR2HSV)

In [6]:
# observation
# The H Component is very similar in both the images which indicates the color information is intact even under illumination changes.
# The S component is also very similar in both images.
# The V Component captures the amount of light falling on it thus it changes due to illumination changes.
# There is drastic difference between the values of the red piece of outdoor and Indoor image. This is because Hue is represented as a circle and red is at the starting angle. So, it may take values between [300, 360] and again [0, 60].

In [7]:
# how to use these color spaces for segmentation
# the simplest way : lets first try to use them to detect the green color from the cube.

#step1 : get the color values for a particular color
#step2: applying threshold for segmentation

In [ ]:
#python
bgr = [40,158,16]
thresh = 40

